In [2]:
import os
os.environ.update({
    'TORCHDYNAMO_DISABLE': '1',
    'PYTORCH_DISABLE_TRITON': '1',
    'TRITON_DISABLE': '1'
})
import torch

In [3]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
d_type = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    max_seq_length = max_seq_length,
    load_in_4bit = load_in_4bit,
)

C:\Users\James\AppData\Local\Programs\Python\Python313\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


W0913 22:57:50.422000 10852 site-packages\torch\distributed\elastic\multiprocessing\redirects.py:29] NOTE: Redirects are currently not supported in Windows or MacOs.


🦥 Unsloth Zoo will now patch everything to make training faster!


C:\Users\James\AppData\Local\Programs\Python\Python313\Lib\site-packages\unsloth_zoo\gradient_checkpointing.py:339: UserWarning: expandable_segments not supported on this platform (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\c10/cuda/CUDAAllocatorConfig.h:35.)
  GPU_BUFFERS = tuple([torch.empty(2*256*2048, dtype = dtype, device = f"{DEVICE_TYPE}:{i}") for i in range(n_gpus)])


==((====))==  Unsloth 2025.9.4: Fast Mistral patching. Transformers: 4.56.1.
   \\   /|    Quadro P4200. Num GPUs = 1. Max memory: 8.0 GB. Platform: Windows.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 6.1. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [4]:
model = FastLanguageModel.get_peft_model(
    model, 
    r = 64,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64*2,
    lora_dropout = 0,
    bias = 'none',
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None
)

Unsloth 2025.9.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [5]:
from datasets import Dataset
import json

with open("processed_emotion_data.json", "r") as f:
    data = json.load(f)

dataset = Dataset.from_list(data)

In [6]:
from unsloth import to_sharegpt

dataset = to_sharegpt(
    dataset=dataset,
    merged_prompt= \
        "[[The user's sentence is {content}.]]"\
        "[[\nThe user's emotion is {emotion}.]]",
    conversation_extension=10,
    output_column_name="emotion",
)

Extending conversations: 100%|████████████████████████████████████████████| 2518/2518 [00:00<00:00, 4467.36 examples/s]


In [7]:
from pprint import pprint
pprint(dataset[1])

{'conversations': [{'from': 'human',
                    'value': "The user's sentence is My baby just took their "
                             "first steps and I'm over the moon!.\n"
                             "The user's emotion is euphoria."},
                   {'from': 'gpt', 'value': 'euphoria'},
                   {'from': 'human',
                    'value': "The user's sentence is The smell of fresh "
                             'flowers fills me with peace..\n'
                             "The user's emotion is serenity."},
                   {'from': 'gpt', 'value': 'serenity'},
                   {'from': 'human',
                    'value': "The user's sentence is The clock shows it is "
                             'noon..\n'
                             "The user's emotion is equanimity."},
                   {'from': 'gpt', 'value': 'equanimity'},
                   {'from': 'human',
                    'value': "The user's sentence is That surprise gift made "
 

In [8]:
from unsloth import standardize_sharegpt
dataset = standardize_sharegpt(dataset)

Unsloth: Standardizing formats (num_proc=12): 100%|████████████████████████| 2518/2518 [00:19<00:00, 129.61 examples/s]


In [9]:
chat_template = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|><|begin_of_text|><|start_header_id|>system<|end_header_id|>

{SYSTEM}<|eot_id|><|start_header_id|>user<|end_header_id|>

{INPUT}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{OUTPUT}<|eot_id|>"""

from unsloth import apply_chat_template

dataset = apply_chat_template(
    dataset=dataset,
    tokenizer=tokenizer,
    chat_template=chat_template,
    default_system_message="Use one word to describe the user's emotion.",
)

Unsloth: We automatically added an EOS token to stop endless generations.
Map: 100%|████████████████████████████████████████████████████████████████| 2518/2518 [00:00<00:00, 3417.76 examples/s]


In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

def tokenize_function(examples):
    return tokenizer(
        examples["text"],
        truncation=True,
        max_length=max_seq_length,
        padding="max_length",
        return_tensors="pt",
    )

tokenized_dataset = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=dataset.column_names,
)

trainer = SFTTrainer(
    model=model,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
    max_seq_length=max_seq_length,
    dataset_num_proc=1,
    packing = False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=10,
        max_steps=60,
        learning_rate=5e-6,
        fp16=not is_bfloat16_supported(),
        bf16=is_bfloat16_supported(),
        logging_steps=1,
        optim="adamw_8bit",
        weight_decay=0.01,
        lr_scheduler_type="linear",
        seed = 3407,
        output_dir="emotion_output",
    )
)

Map: 100%|█████████████████████████████████████████████████████████████████| 2518/2518 [00:04<00:00, 577.11 examples/s]


In [ ]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 2,518 | Num Epochs = 1 | Total steps = 60
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 167,772,160 of 7,415,795,712 (2.26% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.590700
2,4.596900
3,4.618500
4,4.631200
5,4.566800


In [ ]:
model.save_pretrained_merged("predict_emotion_model", tokenizer, save_method = "merged_16bit",)